# Part I. Root-finding. Newton's iteration.

Write a function which performs Newton's iteration for a given function $f(x)$ with known derivative $f'(x)$. Your function should find the root of $f(x)$ with a predefined absolute accuracy $\epsilon$. 

In [1]:
import numpy as np


def newton_iteration(f, fder, x0, eps=1e-5, maxiter=1000, m=1, verbose=False):
    """Find a root of $f(x) = 0$ via Newton's iteration starting from x0.
    NOTE: It is modified version, in default version m = 1
    Parameters
    ----------
    f : callable
        The function to find a root of.
    fder : callable
        The derivative of `f`.
    m : float
        multuplication coefficient
    x0 : float
        Initial value for the Newton's iteration.
    eps : float
        The target accuracy. 
        The iteration stops when the distance between successive iterates is below `eps`.
        Default is 1e-5.
    maxiter : int
        The maximum number of iterations (default is 1000.)
        Iterations terminate if the number of iterations exceeds `maxiter`.
        This parameter is only needed to avoid infinite loops if iterations wander off.
    verbose : bool
        if true function prints onformation about process of convergence
    Returns
    -------
    x : float
        The estimate for the root.
    niter : int
        The number of iterations.
    """
    niter = 0
    x = x0
    target = np.abs(f(x) - 0)

    while target > eps and niter < maxiter:
        x = x0 - m*(f(x0) / fder(x0))
        if verbose:
            print(f"x={x} | x0={x0} | f(x0)={f(x0)} | fdef(x0)={fder(x0)}")
        target = round(np.abs(f(x)), 10)
        x0 = x
        niter += 1

    return x0, niter

### Test I.1 

Test your implementation on a simple example, $f(x) = x^2 - 1$ or similar. (20% of the total grade)

In [2]:
f1 = lambda x: x**2 - 1
f1der = lambda x: x*2
x0 = 20
fder = 40
newton_iteration(f1, f1der, x0)

(1.0000000000149214, 8)

### Test I.2

Now consider a function which has a multiple root. Take $f(x) = (x^2 - 1)^2$ as an example. Implement a modified Newton's iteraion,

$$
x_{n+1} = x_{n} - m \frac{f(x_n)}{f'(x_n)}
$$

and vary $m= 1, 2, 3, 4, 5$. Check the number of iterations required for convergence within a fixed $\epsilon$. Are your observations consistent with the expectation that the convergence is quadratic is $m$ equals the multiplicity of the root, and is linear otherwise? (40% of the total grade)

In [3]:
# ... ENTER YOUR CODE HERE ...
m = [1, 2, 3, 4, 5]

f2 = lambda x: (x**2 - 1)**2
f2_der = lambda x: 4*x*(x**2 - 1)
x0 = 1000

for m_coef in m:
    try:
        print(f"[{m_coef}]: {newton_iteration(f2, f2_der, x0, m=m_coef, maxiter=10000)}")
    except Exception as exc:
        print(f"[{m_coef}]: {exc}")

[1]: (1.001021915064901, 33)
[2]: (1.0005538710539448, 12)
[3]: (0.9988477850363489, 12)
[4]: (1000.0000000236469, 10000)
[5]: (0.9989375299429142, 5416)


In [4]:
newton_iteration(f2, f2_der, x0, m=4, maxiter=10000, verbose=True)

Streaming output truncated to the last 5000 lines.
x=1000.0000000236469 | x0=0.0009999999999763531 | f(x0)=0.9999980000010001 | fdef(x0)=-0.003999995999905413
x=0.0009999999999763531 | x0=1000.0000000236469 | f(x0)=999998000095.5874 | fdef(x0)=3999996000.2837625
x=1000.0000000236469 | x0=0.0009999999999763531 | f(x0)=0.9999980000010001 | fdef(x0)=-0.003999995999905413
x=0.0009999999999763531 | x0=1000.0000000236469 | f(x0)=999998000095.5874 | fdef(x0)=3999996000.2837625
x=1000.0000000236469 | x0=0.0009999999999763531 | f(x0)=0.9999980000010001 | fdef(x0)=-0.003999995999905413
x=0.0009999999999763531 | x0=1000.0000000236469 | f(x0)=999998000095.5874 | fdef(x0)=3999996000.2837625
x=1000.0000000236469 | x0=0.0009999999999763531 | f(x0)=0.9999980000010001 | fdef(x0)=-0.003999995999905413
x=0.0009999999999763531 | x0=1000.0000000236469 | f(x0)=999998000095.5874 | fdef(x0)=3999996000.2837625
x=1000.0000000236469 | x0=0.0009999999999763531 | f(x0)=0.9999980000010001 | fdef(x0)=-0.003999995999

(1000.0000000236469, 10000)

## Summary

From experiments above i've noticed that, with larger $m$ and small denominator gives us an explosion of $$ m \frac{f(x_n)}{f'(x_n)} $$
so, with smaller m solution convergeces slower, nevertheless it becomes robust

# Part II. Fixed-point iteration

Consider the following equation:

$$
\sqrt{x} = \cos{x}
$$

Plot the left-hand side and right-hand side of this equation, and localize the root graphically. Estimate the location of the root by visual inspection of the plot.

Write a function which finds the solution using fixed-point iteration up to a predefined accuracy $\epsilon$. Compare the result to an estimate from a visual inspection.

Next, rewrite the fixed-point problem in the form

$$
x = x - \alpha f(x)
$$

where $\alpha$ is the free parameter. Check the dependence of the number of iterations required for a given $\epsilon$ on $\alpha$. Compare your results to an expectation that the optimal value of $\alpha$ is given by 

$$
\alpha = \frac{2}{m + M}
$$

where $0 < m < |f'(x)| < M$ over the localization interval. (40% of the total grade)

In [6]:
import math

func = lambda x: np.cos(math.degrees(x)) - np.sqrt(x)

def fixed_point(func, x0, alpha, eps=1e-5, max_iter=10000, verbose=False):
    """
        Outputs:
            counter: int
                iteration number
            x: float
                x argument
    """
    x = x0
    target = lambda x: np.abs(func(x))
    counter = 0
    while target(x) > eps and counter < max_iter:
        if verbose:
            print(f"[{counter}]: f({x}) = {func(x)} | abs error: {target(x)}")
        x = x0 + alpha*func(x0)
        x0 = x
        counter += 1
    return counter, x

x0 = 1
alpha = 0.01
fixed_point(func, x0, alpha)

(111, 0.9888115947299172)

# Part III. Newton's fractal.

(Not graded). 

Consider the equation

$$
x^3 = 1
$$

It has three solutions in the complex plane, $x_k = \exp(i\, 2\pi k/ 3)$, $k = 0, 1, 2$.

The Newton's iterations converge to one of these solutions, depending on the starting point in the complex plane (to converge to a complex-valued solution, the iteration needs a complex-valued starting point).

Plot the \emph{basins of attraction} of these roots on the complex plane of $x$ (i.e., on the plane $\mathrm{Re}x$ -- $\mathrm{Im}x$). To this end, make a series of calculations, varying the initial conditions on a grid of points. 
Color the grid in three colors, according to the root, to which iterations converged.